In [1]:
import os
import json
import numpy as np
from sklearn.model_selection import cross_validate
import tensorflow.keras as keras
import tensorflow as tf
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_validate
import pandas as pd

In [2]:
df = pd.read_csv('../data/raw/sample_meta.csv')

In [3]:
df.shape

(1890, 27)

In [4]:
test_set = np.array(df.index[df['fold'] == 'test'])

In [5]:
# function to return the cv splits by index from sample_meta.csv

def fetch_cv_indicies(df):
    unique_folds = list(df.fold.unique())
    unique_folds.remove('test')
    # print(unique_folds)
    
    cv_splits = []
    
    for fold in unique_folds:
               
        left = np.array(df.index[df['fold'] == fold])
        
        right = np.array(df.index[(df['fold'] != fold) & (df['fold'] != 'test')])
        
        #print(f'Fold: {fold}\nLeft: {len(left)}\nRight: {len(right)}\nTotal: {(len(left)+len(right))}\n\n')
        
        split = (left, right)
        
        cv_splits.append(split)
        
    return cv_splits



In [6]:
cv_splits = fetch_cv_indicies(df)

In [7]:
mfcc_path = '../data/processed/13_mfcc_5.json'

In [8]:
def load_data(data_path, target_labels):
    
    with open(data_path, 'r') as fp:
        data = json.load(fp)
        
    # convert lists to numpy arrays
    X = np.array(data['mfcc'])
    y = np.array(data[target_labels])

    print('Data succesfully loaded!')

    return  X, y

In [9]:
# load data
X, y = load_data(data_path=mfcc_path, target_labels='subclass_label')

Data succesfully loaded!


In [10]:
def build_model():
    model = keras.Sequential([

        # input layer
        keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),

        # 1st dense layer
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.2),

        # 2nd dense layer
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.2),

        # 3rd dense layer
        keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.2),

        # output layer
        keras.layers.Dense(5, activation='softmax')
    ])
    
    # compile model
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [11]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.FATAL)


In [14]:
# evaluate model with cv splits
build = build_model()
model = KerasClassifier(model=build, batch_size=32, epochs=50, verbose=1)

results = cross_validate(model, X, y, cv=cv_splits)
print("Baseline: %.2f%% (%.2f%%)" % (results['test_score'].mean()*100, results['test_score'].std()*100))



Epoch 1/50
6/6 [==============================] - 1s 24ms/step - loss: 149.2883 - accuracy: 0.1494
Epoch 2/50
6/6 [==============================] - 0s 26ms/step - loss: 69.8534 - accuracy: 0.0287
Epoch 3/50
6/6 [==============================] - 0s 26ms/step - loss: 76.6663 - accuracy: 0.0345
Epoch 4/50
6/6 [==============================] - 0s 27ms/step - loss: 53.4447 - accuracy: 0.0862
Epoch 5/50
6/6 [==============================] - 0s 27ms/step - loss: 56.1992 - accuracy: 0.1149
Epoch 6/50
6/6 [==============================] - 0s 28ms/step - loss: 45.9218 - accuracy: 0.0575
Epoch 7/50
6/6 [==============================] - 0s 27ms/step - loss: 27.7472 - accuracy: 0.0402
Epoch 8/50
6/6 [==============================] - 0s 30ms/step - loss: 40.3076 - accuracy: 0.1207
Epoch 9/50
6/6 [==============================] - 0s 34ms/step - loss: 29.6790 - accuracy: 0.0747
Epoch 10/50
6/6 [==============================] - 0s 32ms/step - loss: 27.6083 - accuracy: 0.0575
Epoch 11/50
6/6 [=

Epoch 1/50
6/6 [==============================] - 1s 26ms/step - loss: 141.5728 - accuracy: 0.1790
Epoch 2/50
6/6 [==============================] - 0s 26ms/step - loss: 115.7716 - accuracy: 0.2469
Epoch 3/50
6/6 [==============================] - 0s 27ms/step - loss: 98.6299 - accuracy: 0.3333
Epoch 4/50
6/6 [==============================] - 0s 26ms/step - loss: 94.9326 - accuracy: 0.1296
Epoch 5/50
6/6 [==============================] - 0s 27ms/step - loss: 88.6997 - accuracy: 0.0309
Epoch 6/50
6/6 [==============================] - 0s 26ms/step - loss: 62.5046 - accuracy: 0.1358
Epoch 7/50
6/6 [==============================] - 0s 27ms/step - loss: 46.8771 - accuracy: 0.2840
Epoch 8/50
6/6 [==============================] - 0s 30ms/step - loss: 48.3743 - accuracy: 0.2963
Epoch 9/50
6/6 [==============================] - 0s 29ms/step - loss: 46.2871 - accuracy: 0.1790
Epoch 10/50
6/6 [==============================] - 0s 31ms/step - loss: 42.1142 - accuracy: 0.1296
Epoch 11/50
6/6 [

Epoch 1/50
7/7 [==============================] - 1s 22ms/step - loss: 93.6089 - accuracy: 0.7360
Epoch 2/50
7/7 [==============================] - 0s 22ms/step - loss: 50.6265 - accuracy: 0.8629
Epoch 3/50
7/7 [==============================] - 0s 24ms/step - loss: 37.1325 - accuracy: 0.8832
Epoch 4/50
7/7 [==============================] - 0s 24ms/step - loss: 33.2475 - accuracy: 0.9492
Epoch 5/50
7/7 [==============================] - 0s 26ms/step - loss: 18.5628 - accuracy: 0.8071
Epoch 6/50
7/7 [==============================] - 0s 24ms/step - loss: 15.6475 - accuracy: 0.8274
Epoch 7/50
7/7 [==============================] - 0s 24ms/step - loss: 16.2672 - accuracy: 0.9036
Epoch 8/50
7/7 [==============================] - 0s 28ms/step - loss: 10.8797 - accuracy: 0.8376
Epoch 9/50
7/7 [==============================] - 0s 30ms/step - loss: 4.7662 - accuracy: 0.8832
Epoch 10/50
7/7 [==============================] - 0s 29ms/step - loss: 11.9639 - accuracy: 0.8223
Epoch 11/50
7/7 [===

Epoch 1/50
6/6 [==============================] - 1s 23ms/step - loss: 97.7554 - accuracy: 0.7152
Epoch 2/50
6/6 [==============================] - 0s 22ms/step - loss: 43.3531 - accuracy: 0.9576
Epoch 3/50
6/6 [==============================] - 0s 22ms/step - loss: 31.5425 - accuracy: 0.9697
Epoch 4/50
6/6 [==============================] - 0s 23ms/step - loss: 25.1178 - accuracy: 0.8909
Epoch 5/50
6/6 [==============================] - 0s 24ms/step - loss: 14.2675 - accuracy: 0.9091
Epoch 6/50
6/6 [==============================] - 0s 28ms/step - loss: 12.0205 - accuracy: 0.9212
Epoch 7/50
6/6 [==============================] - 0s 28ms/step - loss: 19.0919 - accuracy: 0.9273
Epoch 8/50
6/6 [==============================] - 0s 29ms/step - loss: 13.2802 - accuracy: 0.9273
Epoch 9/50
6/6 [==============================] - 0s 29ms/step - loss: 7.5227 - accuracy: 0.9152
Epoch 10/50
6/6 [==============================] - 0s 31ms/step - loss: 6.2592 - accuracy: 0.9152
Epoch 11/50
6/6 [====

Epoch 1/50
6/6 [==============================] - 1s 22ms/step - loss: 91.8045 - accuracy: 0.5932
Epoch 2/50
6/6 [==============================] - 0s 21ms/step - loss: 42.4837 - accuracy: 0.6158
Epoch 3/50
6/6 [==============================] - 0s 23ms/step - loss: 34.2448 - accuracy: 0.6497
Epoch 4/50
6/6 [==============================] - 0s 23ms/step - loss: 32.4755 - accuracy: 0.7797
Epoch 5/50
6/6 [==============================] - 0s 24ms/step - loss: 25.6655 - accuracy: 0.6384
Epoch 6/50
6/6 [==============================] - 0s 24ms/step - loss: 19.3281 - accuracy: 0.5763
Epoch 7/50
6/6 [==============================] - 0s 25ms/step - loss: 15.0625 - accuracy: 0.7175
Epoch 8/50
6/6 [==============================] - 0s 25ms/step - loss: 13.0772 - accuracy: 0.6158
Epoch 9/50
6/6 [==============================] - 0s 26ms/step - loss: 10.1850 - accuracy: 0.6328
Epoch 10/50
6/6 [==============================] - 0s 27ms/step - loss: 11.7691 - accuracy: 0.6780
Epoch 11/50
6/6 [==

Epoch 1/50
5/5 [==============================] - 1s 29ms/step - loss: 129.7173 - accuracy: 0.6266
Epoch 2/50
5/5 [==============================] - 0s 29ms/step - loss: 47.1291 - accuracy: 0.8165
Epoch 3/50
5/5 [==============================] - 0s 33ms/step - loss: 34.9336 - accuracy: 0.6139
Epoch 4/50
5/5 [==============================] - 0s 29ms/step - loss: 29.1314 - accuracy: 0.7468
Epoch 5/50
5/5 [==============================] - 0s 28ms/step - loss: 15.5157 - accuracy: 0.8228
Epoch 6/50
5/5 [==============================] - 0s 29ms/step - loss: 12.7559 - accuracy: 0.7658
Epoch 7/50
5/5 [==============================] - 0s 30ms/step - loss: 12.7573 - accuracy: 0.7025
Epoch 8/50
5/5 [==============================] - 0s 32ms/step - loss: 7.8105 - accuracy: 0.7025
Epoch 9/50
5/5 [==============================] - 0s 32ms/step - loss: 8.1526 - accuracy: 0.7658
Epoch 10/50
5/5 [==============================] - 0s 36ms/step - loss: 15.2845 - accuracy: 0.7911
Epoch 11/50
5/5 [===

Epoch 1/50
6/6 [==============================] - 1s 26ms/step - loss: 59.2375 - accuracy: 0.0904
Epoch 2/50
6/6 [==============================] - 0s 30ms/step - loss: 39.9184 - accuracy: 0.0106
Epoch 3/50
6/6 [==============================] - 0s 30ms/step - loss: 37.3135 - accuracy: 0.0160
Epoch 4/50
6/6 [==============================] - 0s 27ms/step - loss: 27.1957 - accuracy: 0.0000e+00
Epoch 5/50
6/6 [==============================] - 0s 32ms/step - loss: 17.8452 - accuracy: 0.0372
Epoch 6/50
6/6 [==============================] - 0s 34ms/step - loss: 21.8823 - accuracy: 0.0479
Epoch 7/50
6/6 [==============================] - 0s 33ms/step - loss: 15.2929 - accuracy: 0.0213
Epoch 8/50
6/6 [==============================] - 0s 34ms/step - loss: 18.5216 - accuracy: 0.0266
Epoch 9/50
6/6 [==============================] - 0s 34ms/step - loss: 12.9478 - accuracy: 0.0532
Epoch 10/50
6/6 [==============================] - 0s 34ms/step - loss: 11.7662 - accuracy: 0.0372
Epoch 11/50
6/6

Epoch 1/50
6/6 [==============================] - 1s 26ms/step - loss: 77.0502 - accuracy: 0.5057
Epoch 2/50
6/6 [==============================] - 0s 30ms/step - loss: 39.1261 - accuracy: 0.5057
Epoch 3/50
6/6 [==============================] - 0s 27ms/step - loss: 15.6782 - accuracy: 0.5402
Epoch 4/50
6/6 [==============================] - 0s 32ms/step - loss: 17.6850 - accuracy: 0.5287
Epoch 5/50
6/6 [==============================] - 0s 30ms/step - loss: 15.5162 - accuracy: 0.5977
Epoch 6/50
6/6 [==============================] - 0s 32ms/step - loss: 10.4562 - accuracy: 0.5575
Epoch 7/50
6/6 [==============================] - 0s 32ms/step - loss: 8.4325 - accuracy: 0.5172
Epoch 8/50
6/6 [==============================] - 0s 32ms/step - loss: 10.2524 - accuracy: 0.5172
Epoch 9/50
6/6 [==============================] - 0s 33ms/step - loss: 7.6403 - accuracy: 0.5402
Epoch 10/50
6/6 [==============================] - 0s 32ms/step - loss: 8.1460 - accuracy: 0.5402
Epoch 11/50
6/6 [=====

Epoch 1/50
6/6 [==============================] - 1s 27ms/step - loss: 88.0706 - accuracy: 0.1792
Epoch 2/50
6/6 [==============================] - 0s 25ms/step - loss: 35.0013 - accuracy: 0.0173
Epoch 3/50
6/6 [==============================] - 0s 27ms/step - loss: 26.8086 - accuracy: 0.0578
Epoch 4/50
6/6 [==============================] - 0s 28ms/step - loss: 21.8070 - accuracy: 0.0751
Epoch 5/50
6/6 [==============================] - 0s 28ms/step - loss: 18.7670 - accuracy: 0.1098
Epoch 6/50
6/6 [==============================] - 0s 30ms/step - loss: 18.5060 - accuracy: 0.0925
Epoch 7/50
6/6 [==============================] - 0s 30ms/step - loss: 18.4831 - accuracy: 0.0578
Epoch 8/50
6/6 [==============================] - 0s 33ms/step - loss: 13.9789 - accuracy: 0.0809
Epoch 9/50
6/6 [==============================] - 0s 32ms/step - loss: 11.6394 - accuracy: 0.0809
Epoch 10/50
6/6 [==============================] - 0s 32ms/step - loss: 10.3958 - accuracy: 0.0983
Epoch 11/50
6/6 [==

Epoch 1/50
5/5 [==============================] - 1s 21ms/step - loss: 86.0761 - accuracy: 0.5517
Epoch 2/50
5/5 [==============================] - 0s 24ms/step - loss: 50.8895 - accuracy: 0.4690
Epoch 3/50
5/5 [==============================] - 0s 23ms/step - loss: 43.2186 - accuracy: 0.3724
Epoch 4/50
5/5 [==============================] - 0s 26ms/step - loss: 24.7801 - accuracy: 0.4828
Epoch 5/50
5/5 [==============================] - 0s 27ms/step - loss: 23.6791 - accuracy: 0.6690
Epoch 6/50
5/5 [==============================] - 0s 28ms/step - loss: 19.5740 - accuracy: 0.5310
Epoch 7/50
5/5 [==============================] - 0s 26ms/step - loss: 15.6462 - accuracy: 0.4759
Epoch 8/50
5/5 [==============================] - 0s 29ms/step - loss: 12.9413 - accuracy: 0.4483
Epoch 9/50
5/5 [==============================] - 0s 31ms/step - loss: 14.7435 - accuracy: 0.4345
Epoch 10/50
5/5 [==============================] - 0s 33ms/step - loss: 11.2005 - accuracy: 0.5172
Epoch 11/50
5/5 [==

In [15]:
results

{'fit_time': array([10.64386439, 10.50147915, 12.02762651, 10.00153804,  9.90983176,
         9.88993716, 11.32176685, 10.55224967, 10.49927568,  9.08361292]),
 'score_time': array([0.52019906, 0.61391115, 0.54936004, 0.60184264, 0.62918186,
        0.69403338, 0.62550211, 0.55040145, 0.5325644 , 0.63528585]),
 'test_score': array([0.42105263, 0.7582205 , 0.87730871, 0.87596899, 0.86653646,
        0.55498392, 0.68459016, 0.66211826, 0.2038961 , 0.75637755])}